<a href="https://colab.research.google.com/github/shahringon98/Hybrid-CNN---LSTM/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Load and normalize data
df = pd.read_csv('CNN11V.csv', parse_dates=['date'])
df.set_index('date', inplace=True)
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# Function to create segments
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

results_df = pd.DataFrame(columns=['IVs', 'RMSE', 'MAE', 'MAPE', 'sMAPE'])
features = [Nubers of Variables]

# Iterate over different combinations of IVs
for idx in range(1, len(features) + 1):
    iv_name = ', '.join(features[:idx])

    # Create segments from the normalized data
    TIME_STEPS = 10
    X, y = create_dataset(df_normalized[features[:idx]], df_normalized['Y'], TIME_STEPS)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Define the hybrid CNN+LSTM model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=0)

    # Generate predictions for the training set
    y_train_pred = model.predict(X_train)

    # Inverse scale the predictions and actual values
    y_train_actual = scaler.inverse_transform(np.concatenate([df_normalized.iloc[TIME_STEPS:len(X_train)+TIME_STEPS, :-1], y_train.reshape(-1, 1)], axis=1))[:, -1]
    y_train_pred_rescaled = scaler.inverse_transform(np.concatenate([df_normalized.iloc[TIME_STEPS:len(X_train)+TIME_STEPS, :-1], y_train_pred], axis=1))[:, -1]

    # Evaluation metrics
    rmse = sqrt(mean_squared_error(y_train_actual, y_train_pred_rescaled))
    mae = mean_absolute_error(y_train_actual, y_train_pred_rescaled)
    mape = np.mean(np.abs((y_train_actual - y_train_pred_rescaled) / y_train_actual)) * 100
    smape = 2.0 * np.mean(np.abs(y_train_actual - y_train_pred_rescaled) / (np.abs(y_train_actual) + np.abs(y_train_pred_rescaled)))

    results_df = results_df.append({
        'IVs': iv_name,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'sMAPE': smape
    }, ignore_index=True)

    # Plot the predictions vs actual values for the training set
    plt.figure(figsize=(12, 6))
    plt.plot(y_train_actual, label='True Values', color='blue')
    plt.plot(y_train_pred_rescaled, label='Predictions', color='red', linestyle='dashed')
    plt.legend()
    plt.title(f'Multivariate Forecasting using IVs: {iv_name}')
    plt.show()

print(results_df)
